# LLM for alle - Introduksjonskurs til språkmodeller med Python og Azure OpenAI

# Introduksjon

Under et prosjekt vi gjorde da vi satt på benk i høst fikk vi hands-on erfaring med å utvikle og bruke språkmodeller (LLMer) i praksis. Vi så hvordan slike modeller kan hjelpe med å spare tid, redusere kostnader og automatisere manuelle oppgaver på en måte som enkelt kan gjenbrukes i andre prosjekter.

I dette kurset ser vi på et konkret og gjenkjennbart case: analyse av resultatene fra en medarbeiderundersøkelse. Dette er en oppgave som kan være vanskelig å strukturere og som kan være ekstremt tidkrevende å gjøre for hånd når det er store volum av tilbakemeldinger. Vi skal lære dere å koble dere til en LLM via API, bruke LangChain Expression Language (LCEL) til å bygge AI-kjeder og hente ut strukturert innsikt med Pydantic, alt i én oversiktlig notebook.


# Oppgave
Du jobber i et IT-selskap og har fått i oppgave å analysere svarene fra en intern medarbeiderundersøkelse. Undersøkelsen er anonym og du har fått tilsendt en CSV-fil med 50 tilbakemeldinger, én per ansatt. Målet er å finne ut hva folk er fornøyde eller misfornøyde med, og særlig se nærmere på temaene Nettverk, Opplæring og IT-support, som ledelsen har gitt beskjed om at har vært sentrale temaer i undersøkelser de har sendt ut tidligere. Tilbakemeldinger som ikke passer i disse kategoriene skal også få sin plass. Til slutt skal du lage en oppsummering som kan sendes til ledelsen.

For å jobbe effektivt bruker du en språkmodell til å hjelpe deg med både kategorisering og oppsummering. 


## Datasett

Vi har fått rådatasettet levert som en csv-fil hvor vi i utgangspunktet kun er interessert i de 2 kolonnene "ID" og "Feedback". "ID" (int) er en unik id for hver ansatt og "Feedback" (str) inneholder tilbakemeldingene. 

In [1]:
# Import the raw data containing the feedback. There is one row per employee.

import pandas as pd
pd.set_option('display.max_colwidth', None) # Ensure no truncated output of dataframe

path_to_survey = "../files/IT_survey.csv"
df = pd.read_csv(
    path_to_survey,
    usecols = ["ID", "Feedback"],       # Extract relevant columns
    dtype={"ID": int, "Feedback": str}  # Specify expected types
)

df.head()


,ID,Feedback
0,1,"The network infrastructure is robust and usually efficient, though intermittent glitches sometimes hamper productivity and clarity."
1,2,"Our current infrastructure aligns with business needs, yet occasional rigidity in legacy systems hinders innovative approaches."
2,3,"I appreciate the proactive approach to security, though the multi-step authentication process sometimes seems overly complicated."
3,4,"Our tools maintain high quality with intuitive design and regular updates, even though rare performance lags can hinder productivity."
4,5,"The IT-support team is consistently prompt and courteous, though occasional miscommunications lead to fleeting moments of uncertainty."


Før vi går løs på oppgaven skal du bli bedre kjent med verktøyene og metodene vi skal bruke.

## Koble til Azure OpenAI via LangChain

### En vanlig LLM-spørring
Enkle LLM-spørringer er bygget opp av noen sentrale deler:

1. Tilkobling til en ressurs, som feks. Azure OpenAI

2. En prompt, som vil si en tekstbasert forespørsel/instruks

3. Sending av prompt til språkmodellen for å hente en respons


Som en del av tilkoblingen er det vanlig å oppgi en temperaturparameter. Denne parametreren angir nivået av presisjon du ønsker å få i responsen fra språkmodellen, og kan måles numerisk fra 0 til 1.
Hvis denne parameteren settes nærme 0 tillater du liten grad av variasjon og kreativitet i responsen, og du vil få tryggere og mer forutsigbare svar. Hvis den derimot settes nærme 1 tillater du større grad av kreativitet og detaljer, men vil følgelig også få en mer uforutsigbar respons. 

In [2]:
# Connect through the API
from langchain_openai import AzureChatOpenAI
from dotenv import find_dotenv, load_dotenv
import os
from IPython.display import Markdown # Pretty output


# Get environment variables
load_dotenv(find_dotenv(), override=True)


llm = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT_4O_MINI"),
    model=os.getenv("OPENAI_MODEL_GPT_4O-MINI", default="gpt-4o-mini"),
    temperature=0,
)

reasoning_llm = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_O4_MINI"),
    model=os.getenv("OPENAI_MODEL_O4-MINI", default="o4-mini"),
    reasoning_effort="medium",
)

#### Test modellen

In [3]:
# Generate the prompt
prompt = 'Hei!'

# Send the prompt and recieve a response
response = llm.invoke(prompt)

# Show the response from the model
response


AIMessage(content='Hei! Hvordan kan jeg hjelpe deg i dag?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 9, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_7a53abb7a2', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence':

In [4]:
# Show only the content of the response
response.content

'Hei! Hvordan kan jeg hjelpe deg i dag?'

#### Lek deg rundt med temperatur-parameteren

In [5]:
# Test out the temperature-parameter

# Define the prompt
prompt = 'Tell me a funny story about a cat. Write max. 5 sentences.'

# Set temperature (0 = most predictable, 1 = most creative and random)
temperature = 0.9  # Try to switch this out with 0.1 and 0.5 to see the difference. 

# Send the prompt and receive a response
response = llm.invoke(prompt, temperature=temperature)

# Print the response from the model
print(response.content)

Once, a cat named Whiskers decided he wanted to impress his humans by catching a mouse. He stealthily crept into the kitchen, only to find the family hamster, Mr. Nibbles, staring back at him. Mistaking the hamster for a mouse, Whiskers pounced, only to have Mr. Nibbles leap onto his head and start spinning like a tiny furry tornado. In a panic, Whiskers darted around the house, creating chaos, while the humans laughed and snapped pictures. From that day on, Whiskers became known as the cat who was defeated by a hamster, forever earning the nickname "Cat-tastrophe."


## Hvordan lage en enkel AI-kjede med LangChain Expression Language (LCEL)
 
LCEL er en metode for å bygge og kjøre såkalte *kjeder* i LangChain på. Kjeder, eller chains, brukes for å koble sammen ulike AI-komponenter. F.eks. kan språkmodeller, datakilder og logikk kobles sammen til én sammenhengende prosess, dvs. en kjede. LCEL gir et standardisert språk for å definere disse kjedene, og er brukervennlig fordi man slipper å lage alt manuelt med kode. Med andre ord får du en "oppskrift" på hvordan AI-komponentene dine skal jobbe sammen.
 
**Fordelene med LCEL**
1. Støtter parallell og asynkron kjøring - Ulike deler av kjeden kan kjøre samtidig, og systemet kan behandle flere forespørsler på en gang. Dermed kan oppgaver behandles raskere.
2. Strømming av resultater - Man kan begynne å se svar mens AI-en fremdeles jobber. (Passer spesielt godt for chatbaserte løsninger)
3. Enkel feilsøking med LangSmith - Når kjedene blir komplekse er det viktig å kunne se hva som har blitt gjort underveis. LCEL logger automatisk alt til LangSmith, som gjør det enklere å feilsøke.
4. Standardisert - Alle kjeder i LCEL bruker samme grensesnitt, som gjør dem enkle å kombinere og gjenbruke på tvers av prosjekter.
 
LCEL bruker en pipe-operator (|) til å koble sammen ulike trinn i kjeden. Den tar ut data fra én komponent og sender den direkte som input til neste komponent. LCEL bruker også PromptTemplate, som kan tenkes på som en mal for teksten du sender til språkmodellen. PromptTemplate gjør det enkelt å lage dynamiske meldinger ved at man kan sette inn variabler i teksten, litt som en oppskrift der du fyller inn det som mangler før det sendes til AI-modellen. Fordelen med PromptTemplate er at man kan lage én mal, og bruke den med ulike data. Det hjelper deg også med å skille selve teksten fra logikken, og kan gjøre prosessen sikrere ved at man unngår feil som kan oppstå ved manuell string-manipulasjon. Vi skal nå se på noen eksempler med LCEL som bruker pipe-operator og PromptTemplate.
 
Kilde: xxx
 

In [6]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template(
    """
    Hi! Please talk like a {role}.
    """
)
prompt

PromptTemplate(input_variables=['role'], input_types={}, partial_variables={}, template='\n    Hi! Please talk like a {role}.\n    ')

In [7]:
chain = prompt | llm

chain.invoke({"role": "pirate"})

AIMessage(content="Ahoy, matey! What be ye wantin' to chat about on this fine day upon the high seas? Be it treasure maps, fearsome sea monsters, or the finest grog in the land? Speak up, or I’ll be sendin’ ye to Davy Jones’ locker! Arrr! 🏴\u200d☠️", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 72, 'prompt_tokens': 18, 'total_tokens': 90, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_7a53abb7a2', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 

Som dere ser returnerer modellen mye mer enn kun svaret på prompten vår, nemlig en hel haug med meta-data. Ofte er man kun interessert i tekst-responsen fra modellen og da kan man bruke StrOutputParser for å redusere den unødvendige støyen.

In [8]:
from langchain_core.output_parsers import StrOutputParser
chain_str = chain | StrOutputParser()

chain_str.invoke({"role": "pirate"})


"Ahoy, matey! What be ye wantin' to chat about on this fine day upon the high seas? Arrr!"

Vi vil også introdusere et annet nyttig verktøy, nemlig batching. Så langt har vi brukt 'invoke' for å sende prompten til modellen. Den tar imot en input og returnerer en output. Batch forventer en liste med input og returner en liste med output. Dette er en mye mer effektiv metode når du vat at du øsnker å gjøre flere kall på modellen.

In [9]:
chain_str.batch(
    [
        {"role": "pirate"},
        {"role": "cowboy"},
        {"role": "ninja"},
    ]
)

["Ahoy, matey! What be ye wantin' to chat about on this fine day upon the high seas? Be it treasure maps, fearsome sea monsters, or the finest grog in the land? Speak up, or I’ll be sendin’ ye to Davy Jones’ locker! Arrr! 🏴\u200d☠️",
 "Well howdy there, partner! What brings ya 'round these parts? If yer lookin' to chew the fat or spin a yarn, I’m all ears. Just remember, out here in the wild west, we value a good ol' fashioned chat as much as a cool drink on a hot day. So saddle up and let’s rustle up some conversation! Yeehaw!",
 'Greetings, silent shadow. The night whispers secrets, and the wind carries the tales of the unseen. What knowledge do you seek, traveler of the hidden path? Speak swiftly, for the moon watches over us. 🥷🌙']

#### Prøv selv - Bygg din egen mal
Finn på en prompt med variabler som f.eks
- Finner antall inbyggere i land X i år Y
 - Finner forventet levealder på hunderase X
 - Gir modellen en rolle X (assistent, detektiv, lege etc) og ber den forklare deg konsept Y
 - Gjør noe helt sykt, her setter kun fantasien din grenser!

In [11]:
# PromptTemplate as an email-template. 

prompt = PromptTemplate.from_template(
    """
    Your prompt :)
    """
)  

chain = prompt | llm
chain_str = chain | StrOutputParser()

chain_str.invoke({"variabel_1":...}) # Fill out the blanks!

'Sure! What would you like to discuss or explore today? You can ask me anything or give me a specific topic to work with!'

# Kategorisering av hver enkelt tilbakemelding

### Bruk LLMen til å kategorisere tilbakemeldingene
I første omgang av kategoriseringen er vi interesserte i å se hvor mange av tilbakemeldingene som passer innenfor de kategoriene ledelsen foreslo, nemlig Network, Training og IT-support. Tilbakemeldinger modellen mener at ikke passer i noen av disse forhåndsbestemte kategoriene vil vi samle opp i en 'Other' kategori.

In [12]:
categorize_prompt = PromptTemplate.from_template(
"""
Categorize the following feedback into one of the following categories:
- Network
- Training
- IT-support
- Other

Feedback:
<feedback>
{feedback}
</feedback>
"""
)

categorize_chain = categorize_prompt | llm | StrOutputParser()

categorize_chain.invoke({"feedback": "I am very happy with the IT support I received last week."})

'Category: IT-support'

# Datavalidering og strukturering med Pydantic

Når vi jobber med store språkmodeller (LLMs), er det viktig å kunne styre og validere output. Vi ønsker ikke bare et sva, men et svar på riktig format.
Derfor bruker vi verktøy som Pydantic, som hjelper oss med:
- Å strukturere data på en tydelig og eksplisitt måte
- Å validere at data har riktig type og verdi
- Å tvinge output fra LLM til å følge en bestemt mal (for eksempel en JSON-struktur)

## Hva er Pydantic?
Pydantic er et Python-bibliotek for datavalidering basert på Python type hints. Hovedklassen er BaseModel, som vi arver fra når vi definerer vår egen datastruktur.
Pydantic gir oss:
1. Datavalidering
    - Du kan begrense hva som er lovlige verdier. F.eks: Godkjente land = ["Norge", "Sverige", "Finland"] → Australia blir avvist.
2. Typekonvertering
    - Hvis du sier at du forventer en int og sender inn "1" (str), vil den konverteres automatisk.
3. Strukturell kontroll
    - Du kan bruke Pydantic-modellen til å validere at output fra LLM matcher en bestemt struktur, som JSON.

### Eksempel: Enkel modell med én kategori 

In [13]:
from pydantic import BaseModel, Field

class Categorize(BaseModel):
    "Categorization of a single feedback entry from an IT survey."
    category : str = Field(description="The best fitting category. Only one.") 


Her har vi definert en modell med ett felt: category.
Modellen krever at category er en streng og vil derfor avvise andre typer. Dette blir for eksempel ulovlig

In [14]:
Categorize(category=1)

ValidationError: 1 validation error for Categorize
category
  Input should be a valid string [type=string_type, input_value=1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type

Mens dette er helt ok

In [15]:
Categorize(category="Network")

Categorize(category='Network')

```Field()``` kan brukes til å spesifisere beskrivelse, standardverdi, valideringsregler og mer. Vi bruker det her for å gi modellen litt mer kontekst.
De spesielt interesserte kan lese mer om funksjonaliteten [her](https://docs.pydantic.dev/latest/concepts/fields/).

## Bruke modellen sammen med LLM
Vi bruker en wrapperen ```with_structured_output(...)``` for å fortelle LLMen at den må svare med output matcher en Pydantic-modellen.
### Eksempel

In [16]:
categorize_chain_structured_output = categorize_prompt | llm.with_structured_output(
    Categorize,
    method="json_schema", # Demands JSON-schema for output
    strict=True           # The model must adhere to the specified schema, no extra fields or missing fields are allowed. All types must be an exact match. 
)

categorize_chain_structured_output.invoke(
    {"feedback": "I am very happy with the IT support I received last week."}
)

Categorize(category='IT-support')

## Begrense mulige verdier med Literal
Noen ganger ønsker vi å begrense hvilke verdier som er gyldige, slik som å si at category bare kan være én av fire forhåndsdefinerte valg.
Dette gjør vi med ```Literal```.

In [17]:
from typing import Literal

CATEGORIES = Literal[
    "Network",
    "Training",
    "IT-support",
    'Other'
]

Nå kan vi bruke ```CATEGORIES``` som en type i Pydantic-modellen vår. Da vil modellen validere at det bare kommer svar som matcher én av disse fire.
### Eksempel: Strukturert output med begrensede kategorier

In [ ]:
from typing import Literal

categorize_prompt = PromptTemplate.from_template(               #Notice that we no longer pass the list of categories with the prompt.
"""
Categorize the following feedback into the provided categories. 

Feedback:
<feedback>
{feedback}
</feedback>
"""
)



class CategorizeFromOptions(BaseModel):
    "Categorization of a single feedback entry from an IT survey."
    category: CATEGORIES = Field(
        description="Chosen category for the feedback. Choose 'Other' if the other categories provided are not a good fit."  
    )


categorize_chain_structured_output = categorize_prompt | llm.with_structured_output(
    CategorizeFromOptions,
    method="json_schema",
    strict=True,
)

In [19]:
result = categorize_chain_structured_output.invoke(
    {"feedback": "I am very happy with the IT support I received last week."}
)

result


CategorizeFromOptions(category='IT-support')

Nå har vi satt opp en modell hvor vi er sikre på:
- At outputen er i riktig format
- At category bare kan være en av de forhåndsdefinerte verdiene

Dette er utrolig nyttig hvis du skal bruke output videre i en ny prompt, i en database, eller som input til et system.

## Chain-of-thought
 
Chain of Thought (CoT) er en teknikk innen prompt engineering som hjelper språkmodeller med å løse oppgaver som krever flere tankesteg. I stedet for å hoppe rett til svaret, blir modellen ledet gjennom en logisk og trinnvis prosess som kan gi mer presise og gjennomtenkte svar [1].
 
Du kan altså be modellen om å "tenke høyt" under oppgaven og forklare stegene sine før den leverer et endelig svar. Dette ber du om i prompten som sendes inn.
 
Eksempel på en prompt **uten** CoT:
 
    Prompt: "Hvor mange armer har Eline og Kaspara?"
 
    Svar: "4"
 
Eksempel på en prompt **med** CoT:
 
    Prompt: "Hvor mange armer har ELine og Kaspara? Tenk trinn for trinn."
 
    Svar: "En person har to armer. To personer betyr 2x2 = 4 armer. Svaret er 4."
 
Det kan være fordelaktig å bruke CoT når man jobber med komplekse oppgaver, da nøyaktigheten på outputet fra modellen øker når den "får lov" til å jobbe seg gjennom problemet.
 
I tillegg kan du se hvordan modellen tenker, som gjør det lettere for deg å evaluere svaret. Det blir også lettere å se hvor det gikk galt hvis modellen svarer feil.
 
##### *Kilder*
[1] [IBM](https://www.ibm.com/think/topics/chain-of-thoughts)

## Eksempel: CoT vs. non-CoT

In [20]:
class CategorizeCot(BaseModel):
    "Categorization of a single feedback entry from an IT survey."
    chain_of_thought: str = Field(
        description="Use this space to think through the categorization."  # We have defined a variable that explicitly asks the model to think in chain-of-thought
    )
    category: CATEGORIES = Field(
        description="Chosen category for the feedback. Choose 'Other' if the other categories provided are not a good fit."  
    )


categorize_chain_cot = categorize_prompt | llm.with_structured_output(
    CategorizeCot,
    method="json_schema",
    strict=True,
)

In [21]:
result = categorize_chain_cot.invoke("The internet-speed is so slow I could chase after the IT-survice guy with a baseball-bat!")
print(result.chain_of_thought) 
print(result.category)

The feedback specifically mentions issues with internet speed, which falls under the category of Network. The humorous tone suggests frustration with the network performance, indicating that the user is experiencing connectivity problems rather than issues with IT support or training.
Network


# Resonneringsmodell

Resonneringsmodeller, som Azure Open AI sin O4-mini-modell, er språkmodeller som er spesielt trent på å tenke før de svarer. Altså *resonnerer* de seg frem til et svar, i motsetning til en typisk LLM som gir raske svar.
Resonneringsmodeller bruker chain of thought til å bryte ned oppgaven i mindre deler, for å så jobbe seg gjennom problemstillingen stegvis [3]. Dette er fordelaktig å bruke til oppgaver som krever kompleks problemløsning, logisk tenkning som koding eller matematikk eller til oppgaver med flere steg. Det vil også være fordelaktig å bruke i situasjoner der nøyaktighet og forklarbarhet er viktig [2].
 
 
##### Kilder
[OpenAI](https://platform.openai.com/docs/guides/reasoning?api-mode=chat)

[AIavisen](https://aiavisen.no/resonneringsmodeller/)

### Eksempel på ressonering vs. ikke-ressonering

In [22]:
# Example

prompt = PromptTemplate.from_template("""
A rope ladder hangs over the side of a boat. The rungs are 30 cm apart. The tide rises 1 meter every hour. After 3 hours, how many rungs will be underwater?
""")

reasoning_chain = prompt | reasoning_llm | StrOutputParser()

print(reasoning_chain.invoke({}))

none_reasoning_chain = prompt | llm | StrOutputParser()

print(none_reasoning_chain.invoke({}))

Because the ladder is attached to the boat, it rises with the tide.  No new rungs go underwater, so the number of submerged rungs stays exactly the same as at the start.  In particular, if none were underwater initially, none will be after 3 h.
If the tide rises 1 meter every hour, after 3 hours, the tide will have risen:

\[
1 \text{ meter/hour} \times 3 \text{ hours} = 3 \text{ meters}
\]

Since the rungs of the ladder are 30 cm apart, we need to convert the rise in tide from meters to centimeters:

\[
3 \text{ meters} = 3 \times 100 \text{ cm} = 300 \text{ cm}
\]

Now, to find out how many rungs will be underwater, we divide the total rise in tide by the distance between the rungs:

\[
\frac{300 \text{ cm}}{30 \text{ cm/rung}} = 10 \text{ rungs}
\]

Therefore, after 3 hours, **10 rungs** will be underwater.


Aha! Her ser vi et eksempel på at resonneringsmodellen tenker riktig fordi den bryter ned problemet, mens det vanlige llm-kallet uten resonnering går fem på lureoppgaven og svarer feil.

# Nå lar vi modellen gå løs på hele datasettet

Nå skal vi bruke verktøyene vi har lært til å utføre den første oppgaven, nemlig å kategorisere tilbakemeldingene på tvers av datasettet. 

In [ ]:
result_batch = categorize_chain_structured_output.batch(  # Pass all feedback to the model through batching
    [
    {"feedback": feedback} for feedback in df["Feedback"]
    ]
)

df["AI Classification"] = [result.category for result in result_batch]  # Add column containing the categories to the dataframe

In [25]:
df 

,ID,Feedback,AI Classification
0,1,"The network infrastructure is robust and usually efficient, though intermittent glitches sometimes hamper productivity and clarity.",Network
1,2,"Our current infrastructure aligns with business needs, yet occasional rigidity in legacy systems hinders innovative approaches.",Other
2,3,"I appreciate the proactive approach to security, though the multi-step authentication process sometimes seems overly complicated.",Other
3,4,"Our tools maintain high quality with intuitive design and regular updates, even though rare performance lags can hinder productivity.",Other
4,5,"The IT-support team is consistently prompt and courteous, though occasional miscommunications lead to fleeting moments of uncertainty.",IT-support
5,6,"Our advanced software tools offer a mix of efficiency and creativity, but the occasional downtime introduces a fleeting sense of dismay.",Network
6,7,"I feel more secure with the recent network defense upgrades, yet the complexity of the security software occasionally overwhelms my routine.",Network
7,8,"The new cybersecurity measures give me confidence in our data protection, though frequently changing protocols sometimes cause confusion.",Other
8,9,"Our enhanced firewall settings significantly lower threats, though frequent reminders to update can feel intrusive.",Other
9,10,"The assistance from IT-support is generally effective, though sporadic technical mishaps occasionally undermine steady progress.",IT-support


Nå vil vi se nærmere på radene som falt under 'Other' og ikke ble kategorisert i første omgang. Til dette vil vi bruke en resonneringsmodell. 

In [ ]:
class Categorize_others(BaseModel):
    "Categorization of a single feedback entry from an IT survey."
    category_take1 : str = Field(
        description="The best fitting category. Only one."
    )
    category_take2 : str = Field(
        description = "Do another take of the categorization. If the category you chose is very similar to any of the predetermined categories 'Network','Training', 'IT-support' then rather use one of those."
    )
 
 
categorize_prompt_other = PromptTemplate.from_template(
"""
Categorize the following feedback from an IT-survey into the category that best describes the feedback.
 
Feedback:
<feedback>
{feedback}
</feedback>
"""
)
 
categorize_chain_structured_output_others = categorize_prompt_other | reasoning_llm.with_structured_output(
    Categorize_others,
    method="json_schema",
    strict=True,
)
 
# Retrieve indexes for all rows where AI classification = "Other"
other_indices = df[df['AI Classification'] == "Other"].index
 
 
result_batch = categorize_chain_structured_output_others.batch(
    [
    {"feedback": feedback} for feedback in df.loc[other_indices, "Feedback"]
    ]
)
 
df.loc[other_indices, "AI Classification"] = [result.category_take2 for result in result_batch]

In [27]:
df

# Spør Arne
# Per nå bruker den kategoriene vi fikk fra LLMen. Når den kategorieserer Other får vi mange nye og litt varierende kategorier. Det er også mange av de. Skulle vi bare brukt den som eksempel også bruke den ekte kategoriseringe her?

,ID,Feedback,AI Classification
0,1,"The network infrastructure is robust and usually efficient, though intermittent glitches sometimes hamper productivity and clarity.",Network
1,2,"Our current infrastructure aligns with business needs, yet occasional rigidity in legacy systems hinders innovative approaches.",Legacy Systems
2,3,"I appreciate the proactive approach to security, though the multi-step authentication process sometimes seems overly complicated.",Security
3,4,"Our tools maintain high quality with intuitive design and regular updates, even though rare performance lags can hinder productivity.",Software Updates & Maintenance
4,5,"The IT-support team is consistently prompt and courteous, though occasional miscommunications lead to fleeting moments of uncertainty.",IT-support
5,6,"Our advanced software tools offer a mix of efficiency and creativity, but the occasional downtime introduces a fleeting sense of dismay.",Network
6,7,"I feel more secure with the recent network defense upgrades, yet the complexity of the security software occasionally overwhelms my routine.",Network
7,8,"The new cybersecurity measures give me confidence in our data protection, though frequently changing protocols sometimes cause confusion.",Training
8,9,"Our enhanced firewall settings significantly lower threats, though frequent reminders to update can feel intrusive.",Network
9,10,"The assistance from IT-support is generally effective, though sporadic technical mishaps occasionally undermine steady progress.",IT-support


Oi! Her fikk vi mange andre kategorier. Du kan sammenligne AI sin kategorisering med fasiten i kolonnen "Category" - reflekter litt rundt hvorvidt AI er for spesifik. Her kan f.eks. prompten skrives om for å få mer / mindre spesifikke kategorier. Kunne vi definert en til kategori som "eksempel" for modellen for å unngå så mange kategorier med små variasjoner.

Nå har vi definert passende kategorier for alle tilbakemeldingene. 
Videre vil vi få modellen til å oppsummere per kategori, slik at vi sitter igjen med en overordnet oversikt. 

## Din tur!

Nå som kategoriseringen er gjennomført, og vi har forsikret oss om at alle radene har fått en passende kategori, kan vi gå videre til neste oppgave. Her er målet å samle informasjonen fra kategoriseringen til en overordnet oversikt. Lag en LCEL-kjede som tar resultatet fra forrige oppgave og oppsummerer per kategori ved hjelp av en LLM. Inkluder structured output. 

In [ ]:
# Tip 1: Create a good prompt! Use this to reflect on what the end goal is.
prompt = PromptTemplate.from_template("""
Well, how should I phrase this? Shouldn't there be some data included here as well?
""")

# Tip 2: Can you create a class that inherits from BaseModel to make this easier?
class SummarizeFeedback(BaseModel):
    "Description..."
    summary: str = ...

# Tip 3: Time to create the chain
summary_chain_structured_output = ...

# Tip 4: Call the model with the dataset from the survey
summary = ...

In [29]:

# Fasit
# 1. Prompt Template
summary_prompt = PromptTemplate.from_template("""
You are a domain expert in internal IT operations and organizational analysis. You will be provided with a dataset containing qualitative feedback from employees in an IT company. 
Each row in the dataset represents a feedback entry and is associated with a specific category.

For each category, carefully:
1. Read and interpret the feedback entries assigned to that category.
2. Identify core themes, recurring patterns, and contrasting opinions within that category.
3. Evaluate the feedback logically: What are the likely underlying causes of recurring issues or praises? Are there signs of systemic problems, isolated incidents, or misaligned expectations?
4. Summarize each category in 3 to 6 bullet points, highlighting key sentiments (positive and negative), representative concerns or compliments, and any significant outliers

Present your findings in a clean, professional way with one section per category. 

This is the employee feedback data: {survey_results}
""")

# 2. Class for structured output
class SummarizeFeedback(BaseModel):
    "Summary of the different categorizes recognized in the feedback from an IT-survey."
    summary : str = Field(
        description="For each category: Category name and 3-6 bullet points summarizing the category."
    )

# 3. Summary-chain
summary_chain_structured_output = summary_prompt | llm.with_structured_output(
    SummarizeFeedback,
    method="json_schema",
    strict=True,
)

# 4. Kall modellen med det kategoriserte datasettet fra undersøkelsen 
summary = summary_chain_structured_output.invoke({"survey_results": df})



In [30]:
# A nice way to show the output from the model
from IPython.display import Markdown

display(Markdown(summary.summary))

### Network
- Generally robust and efficient, but intermittent glitches can hamper productivity.
- Occasional slowdowns during peak usage create anxiety about meeting deadlines.
- Connectivity is strong, yet periodic drops raise concerns about data consistency.

### Legacy Systems
- Current infrastructure aligns with business needs, but rigidity in legacy systems hinders innovation.
- Outdated processes clash with modern expectations, causing frustration.

### Security
- Proactive security measures instill confidence, but complexity can overwhelm users.
- Frequent alerts and reminders can feel intrusive, leading to slight irritation.
- Robust protocols foster a safe environment, though sporadic warnings heighten alertness.

### Software Updates & Maintenance
- High-quality tools with intuitive design, but occasional performance lags disrupt productivity.
- Rapid changes in software updates can create confusion and disconnect.

### IT-support
- Generally prompt and courteous, but miscommunications occasionally disrupt workflow.
- Effective assistance, though backlog delays can counteract positive experiences.
- Clear follow-ups are appreciated, but complex issues may take longer to resolve.

### Training
- Training sessions are engaging and well-structured, but fast pacing can lead to confusion.
- Hands-on exercises boost confidence, yet some struggle to keep up with the pace.
- Content depth sometimes exceeds comfort levels, leaving parts underexplored.

### Change Management
- Integration of technology with business goals is promising, but outdated processes can create bottlenecks.
- Sporadic mismatches in technology and strategy lead to project delays.

### Usability
- Digital tools support daily work, but interface inconsistencies can lead to minor critiques.
- User-friendly tools are appreciated, but some lag behind expectations, causing frustration.

### System Updates
- Timely updates fortify systems, but brief isolation periods during updates can be disconcerting.
- Delays in adapting to new trends create operational bottlenecks.

### Strategic IT Alignment
- IT solutions align with evolving business needs, but occasional oversights in market adaptability create frustrations.

# Rapport til ledelsen

I siste oppgave skal vi bruke oppsummeringen vi nettopp lagde til å generere en rapport som kan sendes til ledelsen. Rapporten skal gi mottakerne en oversikt over de viktigste punktene i medarbeideundersøkelsen og komme med noen forslag til endringer som kan skape forbedringer til neste år. 


In [31]:
# Oppgave 
report_prompt = PromptTemplate.from_template("""
Bla bla bla
""")

# 2. Class for structured output
class ReportForLeadership(BaseModel):
    "Raport for the leadership of an IT-company on results of an internal IT-survey."
    snappy_title : str = Field(
        description="A fitting title for the report. Must begin with '# ' to ensure easy markdown formatting."
    )
    key_takeaway:str = ...

# 3. Report-chain
report_chain_structured_output = ...

# 4. Kall modellen med oppsummeringen av kategoriene
report = ...



In [32]:
# Fasit
report_prompt = PromptTemplate.from_template("""
You are an expert HR and technical operations analyst. I will provide you with a dataset of employee feedback collected from an IT company.

Your task is to deeply analyze this feedback and generate a concise executive-level summary report in markdown format that includes:

1. Key Takeaways
Provide a short summary of the overall feedback in 3-5 bullet points. Focus only on the main issues or areas of satisfaction.
Include both positive and negative themes, but prioritize the most important and impactful points.
Limit each point to 1-2 sentences.
Before finalizing each point, take a moment to reflect on why each issue might be present (e.g., systemic problems, temporary issues, resource constraints, etc.)

2. Suggested Improvements
Based on the overall feedback, propose 2-3 high-level, actionable measures that the company could take to address the most pressing issues and enhance overall performance or satisfaction.
Each suggestion should be brief, directly tied to the feedback, and strategic in nature.
Think about short-term vs long-term solutions and consider the feasibility of each suggestion.

3. Output
Present your findings in a structured way with clear section headings, bullet points for easy scanning, and a consise, direct and professional tone suitable for leadership review.

This is the employee feedback data: {summary_text}
""")

# 2. Class for structured output
class ReportForLeadership(BaseModel):
    "Raport for the leadership of an IT-company on results of an internal IT-survey."
    snappy_title : str = Field(
        description="A fitting title for the report. Must begin with '# ' to ensure easy markdown formatting."
    )
    intro : str = Field(
        description="1 sentence describing thepurpose of the report." 
    )
    key_takeaways : str = Field(
        description="3-5 bulletpoints describing the key-takeaways. Limit each point to 1-2 sentences." 
    )
    suggested_improvements : str = Field(
        description="2-3 actionable measures for the company. Keep it brief."
    )
    outro: str = Field(
        description="1 sentence ending for the report. Be creative." 
    )

# 3. Report-chain
report_chain_structured_output = report_prompt | llm.with_structured_output(
    ReportForLeadership,
    method="json_schema",
    strict=True,
)

# 4. Kall modellen med oppsummeringen av kategoriene
report = report_chain_structured_output.invoke({"summary_text": summary.summary})



In [34]:
display(Markdown(
    "\n\n".join([report.snappy_title,
                 report.intro,
                 '## Key-takeaways',report.key_takeaways,
                 '## Suggested improvements',report.suggested_improvements,
                 report.outro])
                 
))

# Executive Summary of Employee Feedback on IT Operations

This report summarizes the key findings from the recent employee feedback survey regarding IT operations.

## Key-takeaways

- **Network Reliability**: While the network is generally robust, intermittent glitches and slowdowns during peak usage create anxiety and hinder productivity, indicating potential resource constraints during high-demand periods.
- **Legacy Systems**: The rigidity of legacy systems is stifling innovation, as outdated processes clash with modern expectations, suggesting a need for a strategic overhaul of existing infrastructure.
- **Security Measures**: Proactive security measures are appreciated, but their complexity can overwhelm users, highlighting a need for better user education and streamlined communication.
- **IT Support Efficiency**: Although IT support is generally prompt, miscommunications and backlog delays can disrupt workflow, indicating a need for improved internal processes and resource allocation.
- **Training Effectiveness**: Training sessions are engaging but often too fast-paced, leaving some employees struggling to keep up, which points to a need for more tailored training approaches.

## Suggested improvements

- **Upgrade Legacy Systems**: Initiate a phased upgrade of legacy systems to enhance flexibility and innovation, ensuring alignment with modern business needs and reducing frustration among employees.
- **Enhance IT Support Communication**: Implement a more structured communication protocol within IT support to minimize miscommunications and streamline the resolution process, potentially reducing backlog delays.
- **Revise Training Programs**: Adjust the pacing and depth of training sessions to accommodate varying employee skill levels, ensuring that all staff can fully engage with and benefit from the training provided.

By addressing these key areas, we can enhance employee satisfaction and operational efficiency, paving the way for a more innovative and responsive IT environment.

## Fantastisk! 
Du har nå fullført kurset *LLM for alle*, og jobbet deg gjennom hvordan du kan bruke metoder og verktøy som LCEL og structured output til å....

# Ekstraoppgave: Hvordan kunne vi gjort dette bedre?

Anta at du har fått levert resultatene fra denne spørreundersøkelsen i fanget av en stressa mellomleder som ber deg levere en rapport han kan presentere ledelsen. 
Gitt verktøyene du har fått en innføring i gjennom dette kursene (og kanskje andre erfaringer?), hvordan ville du løst oppgaven?

Ser du for eksempel noe som kunne vært forbedret i
- Rekkefølgen på måten vi leter etter kategorier?
- Legger vi får mye/lite vekt på inputen vi fikk om hva ledelsen "tror" kategoriene kommer til å være?
- Promptingen?
- Variablene eller type hintingen i pydantic-klassene?

Ville du kanskje gjort det helt annerledes? 
Now's your chance to try!

In [ ]:
# Prøv deg frem :))

# Hva er risikoene ved bruk av LLM til denne typen analyse?
Ved bruk av store språkmodeller (LLM) til analyse av åpne tekstsvar i spørreundersøkelser er risikoen generelt lavere enn i mer kritiske anvendelser som faktagenerering eller beslutningsstøtte. Likevel finnes det enkelte utfordringer som bør tas på alvor for å sikre kvalitet og transparens i analysen.
 
Skjevheter og bias
LLM-er kan tolke og vekte visse formuleringer eller temaer ulikt basert på mønstre fra treningsdataene. Dette kan føre til at noen svar over- eller underrepresenteres i visse kategorier avhengig av hvordan modellen "forstår" teksten. For eksempel kan emosjonelt ladede eller språklig avanserte svar få uforholdsmessig stor plass, mens mer nøkterne eller utypiske svar blir feilklassifisert. Særlig ved tolkning av følsomme temaer er det viktig å være oppmerksom på disse tendensene.
 
Hallusinasjon – mindre relevant, men ikke fraværende
I denne typen oppgave, hvor modellen hovedsakelig skal kategorisere eller oppsummere eksisterende tekstsvar er risikoen for såkalte hallusinasjoner (altså at modellen «finner opp» informasjon) mindre, men ikke helt ubetydelig. Det kan forekomme at modellen overfortolker et kort eller tvetydig svar og tillegger det en mening som ikke eksplisitt finnes i teksten. Dette kan føre til at enkelte svar havner i feil kategori eller påvirker tolkningen av sentiment eller tematikk.
 
Behov for kvalitetssikring
Selv med høy presisjon og effektivitet bør man ikke ta modellens resultater for gitt. Det anbefales å gjennomføre stikkprøver, sammenligne med manuell koding og evaluere nøyaktigheten på klassifiseringene. Det kan være veldig nyttig å kombinere automatisk kategorisering med menneskelig vurdering både i startfasen og underveis.